# Data exploration and preprocessing for titanic dataset
This notebook explores the dataset and looks into:
1. Finding and filling missing values
2. Transform categorical variables into numerical ones
3. Dropping columns that lack predictive power

In [163]:
import pandas as pd
from random import randint


## Data exploration

In [164]:
# Analyze structure of data and columns
titanic_df = pd.read_csv("https://raw.githubusercontent.com/ID2223KTH/id2223kth.github.io/master/assignments/lab1/titanic.csv")
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [165]:
titanic_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

## 1. Missing values
Based on the code below, the titanic dataset features missing values in the following columns:
1. Age
Input random values between the minimum and maximum age range of passengers
2. Cabin
However, 'Cabin' column has so many missing values that it is not reliable or representative as a variable. Will drop the column in a later stage.
3. Embarked
Since only two values are missing here, input the most common port from which the people embarked.



In [166]:
# find NaN values
hasNaNs = []

for column in titanic_df.columns:
  print(column + ": " + str(sum(titanic_df[column].isna())) + ", type: "+ str(titanic_df[column].dtype))
  if sum(titanic_df[column].isna()) > 0:
      hasNaNs.append(column)

PassengerId: 0, type: int64
Survived: 0, type: int64
Pclass: 0, type: int64
Name: 0, type: object
Sex: 0, type: object
Age: 177, type: float64
SibSp: 0, type: int64
Parch: 0, type: int64
Ticket: 0, type: object
Fare: 0, type: float64
Cabin: 687, type: object
Embarked: 2, type: object


Look at values for the columns

In [167]:
# See structure of data
for col in hasNaNs:
    print(col + "\n" + str(titanic_df[col].unique()) + "\n\n")

Age
[22.   38.   26.   35.     nan 54.    2.   27.   14.    4.   58.   20.
 39.   55.   31.   34.   15.   28.    8.   19.   40.   66.   42.   21.
 18.    3.    7.   49.   29.   65.   28.5   5.   11.   45.   17.   32.
 16.   25.    0.83 30.   33.   23.   24.   46.   59.   71.   37.   47.
 14.5  70.5  32.5  12.    9.   36.5  51.   55.5  40.5  44.    1.   61.
 56.   50.   36.   45.5  20.5  62.   41.   52.   63.   23.5   0.92 43.
 60.   10.   64.   13.   48.    0.75 53.   57.   80.   70.   24.5   6.
  0.67 30.5   0.42 34.5  74.  ]


Cabin
[nan 'C85' 'C123' 'E46' 'G6' 'C103' 'D56' 'A6' 'C23 C25 C27' 'B78' 'D33'
 'B30' 'C52' 'B28' 'C83' 'F33' 'F G73' 'E31' 'A5' 'D10 D12' 'D26' 'C110'
 'B58 B60' 'E101' 'F E69' 'D47' 'B86' 'F2' 'C2' 'E33' 'B19' 'A7' 'C49'
 'F4' 'A32' 'B4' 'B80' 'A31' 'D36' 'D15' 'C93' 'C78' 'D35' 'C87' 'B77'
 'E67' 'B94' 'C125' 'C99' 'C118' 'D7' 'A19' 'B49' 'D' 'C22 C26' 'C106'
 'C65' 'E36' 'C54' 'B57 B59 B63 B66' 'C7' 'E34' 'C32' 'B18' 'C124' 'C91'
 'E40' 'T' 'C128' 'D37' 'B3

### Fill age column

In [168]:
col = "Age"
min_age = int(min(titanic_df[col]))
max_age = int(max(titanic_df[col]))
nanSpots = titanic_df[col].isna()
temp = titanic_df[col]
for i, nan in enumerate(nanSpots):
    if nan == True:
        temp[i] = randint(min_age, max_age)
titanic_df[col] = temp

C:\Users\alexa\AppData\Local\Temp\ipykernel_3676\1046380638.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[i] = randint(min_age, max_age)


### Fill embarked

In [169]:
titanic_df["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [170]:
# Find distribution of 'Embarked'
titanic_df["Embarked"] = titanic_df["Embarked"].fillna('S')
titanic_df["Embarked"].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

## 2. Transform categorical variables
The columns 'Sex', 'Embarked' and 'Pclass' are considered as categorical variables.

Firstly, 'Sex' is a binary variable where it attains the value of either male or female which will be encoded by a boolean.

Secondly, the values of 'Embarked' and 'Pclass' will be encoded using one-hot encoding for each.

In [171]:
# binarize gender
titanic_df['Sex'] = titanic_df['Sex'].map({'male': 1, 'female': 0})

# one hot for embark and pclass
cols = ['Embarked', 'Pclass']
for col in cols:
    onehot = pd.get_dummies(titanic_df[col], prefix=col, dtype='int64')
    titanic_df = titanic_df.drop(col, axis=1)
    titanic_df = titanic_df.join(onehot)
titanic_df.head()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,1,0,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,0,0,1,0,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1,0,0,1,0,0
2,3,1,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0,1,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0,0,1,1,0,0
4,5,0,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,0,0,1,0,0,1


## 3. Drop columns with lacking predictive power
Name, passengerId and ticket hold no interpretable information. Therefore, these rows will be dropped.

In addition, the cabin column will also be removed. As earlier mentioned, this column has too many missing values to be reliable.

In [172]:
drop_cols = ['Name', 'PassengerId', 'Ticket', 'Cabin']
titanic_df = titanic_df.drop(['Name', 'PassengerId', 'Ticket', 'Cabin'], axis=1)

In [173]:
titanic_df.corr()

,Survived,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
Survived,1.000000,-0.543351,-0.055740,-0.035322,0.081629,0.257307,0.168240,0.003650,-0.149683,0.285904,0.093349,-0.322308
Sex,-0.543351,1.000000,0.041276,-0.114631,-0.245489,-0.182333,-0.082853,-0.074115,0.119224,-0.098013,-0.064746,0.137143
Age,-0.055740,0.041276,1.000000,-0.175749,-0.170418,0.044140,0.033256,0.059113,-0.066334,0.223961,-0.058537,-0.145325
SibSp,-0.035322,-0.114631,-0.175749,1.000000,0.414838,0.159651,-0.059528,-0.026354,0.068734,-0.054582,-0.055932,0.092548
Parch,0.081629,-0.245489,-0.170418,0.414838,1.000000,0.216225,-0.011069,-0.081228,0.060814,-0.017633,-0.000734,0.015790
Fare,0.257307,-0.182333,0.044140,0.159651,0.216225,1.000000,0.269335,-0.117216,-0.162184,0.591711,-0.118557,-0.413333
Embarked_C,0.168240,-0.082853,0.033256,-0.059528,-0.011069,0.269335,1.000000,-0.148258,-0.782742,0.296423,-0.125416,-0.153329
Embarked_Q,0.003650,-0.074115,0.059113,-0.026354,-0.081228,-0.117216,-0.148258,1.000000,-0.499421,-0.155342,-0.127301,0.237449
Embarked_S,-0.149683,0.119224,-0.066334,0.068734,0.060814,-0.162184,-0.782742,-0.499421,1.000000,-0.161921,0.189980,-0.015104
Pclass_1,0.285904,-0.098013,0.223961,-0.054582,-0.017633,0.591711,0.296423,-0.155342,-0.161921,1.000000,-0.288585,-0.626738


In [174]:
titanic_df = titanic_df.rename(columns=str.lower)
titanic_df.survived = titanic_df.survived.astype(bool)
titanic_df.head()

,survived,sex,age,sibsp,parch,fare,embarked_c,embarked_q,embarked_s,pclass_1,pclass_2,pclass_3
0,0,1,22.0,1,0,7.2500,0,0,1,0,0,1
1,1,0,38.0,1,0,71.2833,1,0,0,1,0,0
2,1,0,26.0,0,0,7.9250,0,0,1,0,0,1
3,1,0,35.0,1,0,53.1000,0,0,1,1,0,0
4,0,1,35.0,0,0,8.0500,0,0,1,0,0,1


In [175]:
titanic_df.to_csv('titanic_preprocessed.csv', index=False)